1. pair representations $h_v^{(0)}$  赋值子图中每个节点和源节点信息
2. Message $M_v^{(1)}$: {$h_v^{(0)}$, 所有指向v的节点x的$h_x$和(x, v)总结后的信息}
3. 总结所有Message

In [1]:
import pandas as pd

In [17]:
sample = pd.read_csv('sample.csv')
cite = pd.read_csv('sample_cite.csv')

In [26]:
# 第一步
src = sample[sample.pid == 10169]
v_df = sample[sample.pid != src.pid.values[0]].reset_index(drop=True)

h_v = pd.DataFrame({
    'src_pid': [src.pid.values[0]] * len(v_df),
    'src_title': [src.title.values[0]] * len(v_df),
    'src_abs': [src['abs'].values[0]] * len(v_df),
    'src_label': [src.label.values[0]] * len(v_df),
    'v_pid': v_df.pid,
    'v_title': v_df.title,
    'v_abs': v_df['abs'],
    'v_label': v_df.label
})
h_v

,src_pid,src_title,src_abs,src_label,v_pid,v_title,v_abs,v_label
0,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,63486,Title: Evaluation and Selection of Biases in M...,"Abstract: In this introduction, we define the ...",Theory
1,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,2440,"Title: Quinlan, 1990 J.R. Quinlan. Learning lo...",Abstract: We describe a ranked-model semantics...,Rule_Learning
2,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,3231,Title: Irrelevant Features and the Subset Sele...,Abstract: We address the problem of finding a ...,Theory
3,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,954315,Title: Inductive Database Design,Abstract: When designing a (deductive) databas...,Rule_Learning
4,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,144330,Title: Utilizing Prior Concepts for Learning,Abstract: The inductive learning problem consi...,Theory
5,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,56115,Title: Improving Tactical Plans with Genetic A...,Abstract:,Genetic_Algorithms
6,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,72908,Title: Applications of a logical discovery engine,Abstract: The clausal discovery engine claudie...,Rule_Learning
7,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,83461,Title: Dlab: A Declarative Language Bias Forma...,Abstract: We describe the principles and funct...,Rule_Learning
8,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,17811,Title: #1 Robust Feature Selection Algorithms,Abstract: Selecting a set of features which is...,Genetic_Algorithms
9,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,37483,Title: Learning Approximate Control Rules Of H...,Abstract: One of the difficult problems in the...,Case_Based
